In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [2]:
CBWs         = [   2,   4,   6]
PBWs         = [  10,  16,  20]
CWNDs        = [  64, 128, 256]
TTIME        = 10
NUMEXP       = 20
TPTALGOs     = ['TCPreno','TCPcubic'] #,'UDP']
ATK_PARA     = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
hostnames    = ['atkr'] + [f'h{i}' for i in range(8)]
iperf_header = ['timestamp','src_addr','src_port','dest_addr','dest_port','transferID','interval','transferred_bytes','bits_per_second']

indices      = [('CBW', CBWs), ('PBW', PBWs), ('CWND', CWNDs), ('TPTALGO', TPTALGOs), ('APC', ATK_PARA)]

basedir   = f'./NUMEXP-{NUMEXP}_TTIME-{TTIME}'

In [3]:
dfall = []
for CBW in CBWs:
    for PBW in PBWs:
        for CWND in CWNDs: 
            for TPTALGO in TPTALGOs:
                for APC in ATK_PARA:
                    for HOST in hostnames:
                        dfi = pd.read_csv(f'{basedir}/CBW-{CBW}/PBW-{PBW}/CWND-{CWND}K/{TPTALGO}/{APC}conn/iperf_{HOST}.csv', names=iperf_header)
                        dfi['host']    = HOST
                        dfi['APC']     = APC
                        dfi['TPTALGO'] = TPTALGO
                        dfi['CWND']    = CWND
                        dfi['PBW']     = PBW
                        dfi['CBW']     = CBW
                        dfall.append(dfi)
                        
dfbigboi = pd.concat(dfall)
dfbigboi

,timestamp,src_addr,src_port,dest_addr,dest_port,transferID,interval,transferred_bytes,bits_per_second,host,APC,TPTALGO,CWND,PBW,CBW
0,20211121150156,10.0.0.1,39988,10.0.0.10,5001,3,0.0-1.0,196664,1573312,atkr,1,TCPreno,64,10,2
1,20211121150157,10.0.0.1,39988,10.0.0.10,5001,3,1.0-2.0,136112,1088896,atkr,1,TCPreno,64,10,2
2,20211121150158,10.0.0.1,39988,10.0.0.10,5001,3,2.0-3.0,136112,1088896,atkr,1,TCPreno,64,10,2
3,20211121150200,10.0.0.1,39988,10.0.0.10,5001,3,3.0-4.0,136112,1088896,atkr,1,TCPreno,64,10,2
4,20211121150201,10.0.0.1,39988,10.0.0.10,5001,3,4.0-5.0,136112,1088896,atkr,1,TCPreno,64,10,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,20211121184334,10.0.0.9,45520,10.0.0.10,5001,3,6.0-7.0,393216,3145728,h7,21,TCPcubic,256,20,6
7,20211121184334,10.0.0.9,45520,10.0.0.10,5001,3,7.0-8.0,393216,3145728,h7,21,TCPcubic,256,20,6
8,20211121184336,10.0.0.9,45520,10.0.0.10,5001,3,8.0-9.0,655360,5242880,h7,21,TCPcubic,256,20,6
9,20211121184336,10.0.0.9,45520,10.0.0.10,5001,3,9.0-10.0,524288,4194304,h7,21,TCPcubic,256,20,6


In [ ]:
SCALAR_FIG_SZ = 5

for indeX in indices:
    for indeY in indices:
        if indices.index(indeX) == indices.index(indeY): continue
        AX_R_NAME, AX_R_VALS = indeX
        AX_C_NAME, AX_C_VALS = indeY
        AX_R_LEN , AX_C_LEN  = len(AX_R_VALS), len(AX_C_VALS)
        print(f'{AX_R_LEN} x {AX_C_LEN}\t', AX_R_NAME, '\tvs.  ', AX_C_NAME)
        
        fig, axs = plt.subplots(AX_R_LEN, AX_C_LEN, figsize=(AX_C_LEN*SCALAR_FIG_SZ, AX_R_LEN*SCALAR_FIG_SZ))
        
        maxY = 0
        for R_VAL in AX_R_VALS:
            df0 = dfbigboi[dfbigboi[AX_R_NAME] == R_VAL]
            for C_VAL in AX_C_VALS:
                df1 = df0[df0[AX_C_NAME] == C_VAL]
                axi = axs[AX_R_VALS.index(R_VAL)][AX_C_VALS.index(C_VAL)]
                
                net_data = [0] * len(range(TTIME))
                for HOST in hostnames:
                    df2 = df1[df1['host'] == HOST]
                    host_data = []
                    for t in range(TTIME):
                        tint = f'{t:.1f}-{t+1:.1f}'
                        df3 = df2[df2['interval'] == tint]
#                         print(HOST, df3.shape)
                        if HOST == 'atkr' and df3.shape[0] > 1:
                            df3 = df3[df3['transferID'] == -1]
#                         print(HOST, df3.shape)
                        host_total = df3.sum()['transferred_bytes'] // df3.shape[0]
#                         if(host_total != host_total):
#                             print(df3)
                        host_data.append(host_total)
                        net_data[t] += host_total
                    maxY = max(maxY, max(host_data), max(net_data))
#                     print(HOST, host_data)
                    axi.plot(range(TTIME), host_data, label=HOST)
                axi.plot(range(TTIME), net_data, label='network')
                axi.legend()
                axi.set_xlabel('time')
                axi.set_ylabel('transferred_bytes')
                axi.set_title(f'{AX_R_NAME} {R_VAL} vs. {AX_C_NAME} {C_VAL}')
                
        for R_VAL in AX_R_VALS:
            for C_VAL in AX_C_VALS:
                axi = axs[AX_R_VALS.index(R_VAL)][AX_C_VALS.index(C_VAL)]
                axi.set_ylim(bottom=0, top=maxY)
                
#         break
#     break

        fig.savefig(f'./VIZTIME/VIZTIME_{AX_R_NAME}-{AX_C_NAME}.png', facecolor="w")

3 x 3	 CBW 	vs.   PBW
3 x 3	 CBW 	vs.   CWND
3 x 2	 CBW 	vs.   TPTALGO
3 x 11	 CBW 	vs.   APC
3 x 3	 PBW 	vs.   CBW
3 x 3	 PBW 	vs.   CWND
3 x 2	 PBW 	vs.   TPTALGO
3 x 11	 PBW 	vs.   APC
3 x 3	 CWND 	vs.   CBW
3 x 3	 CWND 	vs.   PBW
3 x 2	 CWND 	vs.   TPTALGO
3 x 11	 CWND 	vs.   APC
2 x 3	 TPTALGO 	vs.   CBW
2 x 3	 TPTALGO 	vs.   PBW
2 x 3	 TPTALGO 	vs.   CWND
2 x 11	 TPTALGO 	vs.   APC
11 x 3	 APC 	vs.   CBW
11 x 3	 APC 	vs.   PBW
